# SpeakUp2.0: Batch Processing for FreeMoCap 

### 0. Import Necessary Packages 

In [8]:
# Cell 1: Importing necessary packages
import logging
import shutil
from pathlib import Path
from typing import Optional, Union
from freemocap.core_processes.export_data.blender_stuff.export_to_blender.export_to_blender import export_to_blender
from freemocap.core_processes.process_motion_capture_videos.process_recording_folder import process_recording_folder
from freemocap.data_layer.recording_models.post_processing_parameter_models import ProcessingParameterModel
from freemocap.data_layer.recording_models.recording_info_model import RecordingInfoModel
from freemocap.system.paths_and_filenames.file_and_folder_names import RECORDING_PARAMETERS_JSON_FILE_NAME
from freemocap.system.paths_and_filenames.path_getters import get_blender_file_path
from freemocap.utilities.save_dictionary_to_json import save_dictionary_to_json

logging.basicConfig(level=logging.INFO)

### 1. Defyining Relevant Directories, Key Variables & Functions 

In [10]:
# Paths for calibration and video files
input_calibration_files = Path(r'Z:/Davide_MultimodalLab/3_DATA_ANALYSIS/1_MOTION_TRACKING/2_Video_Calibration/calibration_results')
input_video_folders= Path(r'Z:\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\1_Video_Segmentation\video_split')
blender_executable_path = Path('C:/Programs/blender/blender.exe')

print('input_calibration_files:', input_calibration_files)  
print('input_video_folders:', input_video_folders)  
print('blender_executable_path:', blender_executable_path)  


# Cell 2: Defining directories, key variables, and functions
def process_recording(
        calibration_file: Path,
        video_folder: Path,
        blender_executable: Optional[Path] = None
):
    """
    Process a recording folder with the given calibration file and videos.
    """
    # Ensure synchronized_videos folder exists in the output directory
    synchronized_videos_path = video_folder / "synchronized_videos"
    if not synchronized_videos_path.exists():
        synchronized_videos_path.mkdir(parents=True, exist_ok=True)

    rec = ProcessingParameterModel()
    rec.recording_info_model = RecordingInfoModel(recording_folder_path=video_folder)
    rec.recording_info_model.calibration_toml_path = calibration_file

    recording_info_dict = rec.dict(exclude={"recording_info_model"})
    Path(rec.recording_info_model.output_data_folder_path).mkdir(parents=True, exist_ok=True)
    save_dictionary_to_json(
        save_path=rec.recording_info_model.output_data_folder_path,
        file_name=RECORDING_PARAMETERS_JSON_FILE_NAME,
        dictionary=recording_info_dict,
    )

    process_recording_folder(recording_processing_parameter_model=rec)

    if blender_executable:
        blender_file_path = get_blender_file_path(recording_folder_path=video_folder)
        logging.info(f"Exporting to Blender: {blender_file_path}")
        export_to_blender(
            recording_folder_path=rec.recording_info_model.output_data_folder_path,
            blender_file_path=blender_file_path,
            blender_exe_path=blender_executable,
        )
    else:
        logging.warning("No blender executable provided. Blender file will not be exported.")

        
        
        


input_calibration_files: Z:\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\2_Video_Calibration\calibration_results
input_video_folders: Z:\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\1_Video_Segmentation\video_split
blender_executable_path: C:\Programs\blender\blender.exe


## 2. Loop Over Input Calibration files and Video folders to load into FreeMoCap

In [11]:
# Cell 3: Working with loops
calibration_files = list(input_calibration_files.glob("*.toml"))

for calibration_file in calibration_files:
    pilot_id = "_".join(calibration_file.stem.split("_")[:3])  # Extract pilot_xx_dx from the filename
    print('current pilot_id: ', pilot_id)
    logging.info(f"Processing calibration file: {calibration_file}")
    
    print('current calibrtion file: ', calibration_file)

    # Find all matching video folders
    matching_folders = [
        folder for folder in input_video_folders.iterdir()
        if folder.is_dir() and pilot_id in folder.name
    ]
    
    print('We found these matching folder: ', matching_folders)

    for video_folder in matching_folders:
        logging.info(f"Processing video folder: {video_folder}")
        process_recording(
            calibration_file=calibration_file,
            video_folder=video_folder,
            #blender_executable=blender_executable_path
        )
        
        print('We will now load this vide folder ',video_folder)

logging.info("All recordings processed!")

current pilot_id:  pilot_01_d2
[2024-10-06T14:14:10.726030] [Δt:89369.724379s] [    INFO] [root] [2323846283:<module>():7] [PID:36168:MainProcess TID:33348:MainThread ] Processing calibration file: Z:\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\2_Video_Calibration\calibration_results\pilot_01_d2_calibration_charuco_calibration_anipose.toml
2024-10-06 14:14:10,726 - root - INFO - Processing calibration file: Z:\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\2_Video_Calibration\calibration_results\pilot_01_d2_calibration_charuco_calibration_anipose.toml
current calibrtion file:  Z:\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\2_Video_Calibration\calibration_results\pilot_01_d2_calibration_charuco_calibration_anipose.toml
We found these matching folder:  []
current pilot_id:  pilot_01_d3
[2024-10-06T14:14:10.935834] [Δt:0.209805s] [    INFO] [root] [2323846283:<module>():7] [PID:36168:MainProcess TID:33348:MainThread ] Processing calibration file: Z:\Davide_Mu

processing video: pilot_04_d2_Speech_Start_SUD_1_Start_cam1.mp4: 100%|██████████| 7284/7284 [14:25<00:00,  8.42frames/s]

[2024-10-06T14:28:37.177918] [Δt:865.193112s] [    INFO] [skellytracker.trackers.base_tracker.video_handler] [video_handler:close():45] [PID:36168:MainProcess TID:33348:MainThread ] video saved to Z:\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\1_Video_Segmentation\video_split\pilot_04_d2_Speech_Start_SUD_1_Start\annotated_videos\pilot_04_d2_Speech_Start_SUD_1_Start_cam1_mediapipe.mp4
2024-10-06 14:28:37,177 - skellytracker.trackers.base_tracker.video_handler - INFO - video saved to Z:\Davide_MultimodalLab\3_DATA_ANALYSIS\1_MOTION_TRACKING\1_Video_Segmentation\video_split\pilot_04_d2_Speech_Start_SUD_1_Start\annotated_videos\pilot_04_d2_Speech_Start_SUD_1_Start_cam1_mediapipe.mp4


[2024-10-06T14:28:41.512862] [Δt:4.334944s] [    INFO] [skellytracker.process_folder_of_videos] [process_folder_of_videos:process_single_video():131] [PID:36168:MainProcess TID:33348:MainThread ] Processing video: pilot_04_d2_Speech_Start_SUD_1_Start_cam2_mediapipe.mp4 with tracker: MediapipeHolisticTracker
2024-10-06 14:28:41,512 - skellytracker.process_folder_of_videos - INFO - Processing video: pilot_04_d2_Speech_Start_SUD_1_Start_cam2_mediapipe.mp4 with tracker: MediapipeHolisticTracker


processing video: pilot_04_d2_Speech_Start_SUD_1_Start_cam2.mp4: 100%|██████████| 7284/7284 [13:49<00:00,  8.78frames/s]

...

c:\ProgramData\Anaconda3\envs\freemocap-env\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[2024-10-06T23:44:04.429399] [Δt:1.611555s] [    INFO] [freemocap.core_processes.post_process_skeleton_data.post_process_skeleton] [post_process_skeleton:run_post_processing_worker():90] [PID:36168:MainProcess TID:33348:MainThread ] Done with gap filling, filtering, and aligning
2024-10-06 23:44:04,429 - freemocap.core_processes.post_process_skeleton_data.post_process_skeleton - INFO - Done with gap filling, filtering, and aligning
[2024-10-06T23:44:04.429399] [Δt:0.000000s] [    INFO] [freemocap.core_processes.process_motion_capture_videos.processing_pipeline_functions.anatomical_data_pipeline_functions] [anatomical_data_pipeline_functions:calculate_anatomical_data():26] [PID:36168:MainProcess TID:33348:MainThread ] Calculating center of mass...
2024-10-06 23:44:04,429 - freemocap.core_processes.process_motion_capture_videos.processing_pipeline_functions.anatomical_data_pipeline_functions - INFO - Calculating center of mass...


c:\ProgramData\Anaconda3\envs\freemocap-env\Lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[2024-10-06T23:44:13.396131] [Δt:8.966733s] [    INFO] [freemocap.core_processes.post_process_skeleton_data.calculate_center_of_mass] [calculate_center_of_mass:calculate_total_body_COM():306] [PID:36168:MainProcess TID:33348:MainThread ] Calculating Total Body Center of Mass
2024-10-06 23:44:13,396 - freemocap.core_processes.post_process_skeleton_data.calculate_center_of_mass - INFO - Calculating Total Body Center of Mass


[2024-10-06T23:44:17.970683] [Δt:4.574552s] [    INFO] [freemocap.core_processes.detecting_things_in_2d_images.mediapipe_stuff.convert_mediapipe_npy_to_csv] [convert_mediapipe_npy_to_csv:convert_mediapipe_npy_to_csv():23] [PID:36168:MainProcess TID:33348:MainThread ] Converting npy data with shape: (6890, 553, 3) into `csv` and smaller `npy` files
2024-10-06 23:44:17,970 - freemocap.core_processes.detecting_things_in_2d_images.mediapipe_stuff.convert_mediapipe_npy_to_csv - INFO - Converting npy data with shape: (6890, 553, 3) into `csv` and smaller `npy` files
[2024-10-06T23:49:01.443005] [Δt:283.473298s] [   DEBUG] [freemocap.core_processes.detecting_things_in_2d_images.mediapipe_stuff.convert_mediapipe_npy_to_csv] [convert_mediapipe_npy_to_csv:convert_mediapipe_npy_to_csv():239] [PID:36168:MainProcess TID:33348:MainThread ] body 3d xyz shape: (6890, 33, 3)
[2024-10-06T23:49:01.443982] [Δt:0.000000s] [   DEBUG] [freemocap.core_processes.detecting_things_in_2d_images.mediapipe_stuff.co